# 8a – Model Monitoring (Normal Conditions)

This notebook deploys the selected production model
and configures SageMaker Model Monitor to track
data quality and input feature drift.


## Import Required Libraries and Initialize AWS Session


In [45]:
!tar -czvf model.tar.gz model.joblib inference.py


model.joblib
inference.py


In [46]:
import sagemaker

sagemaker_session = sagemaker.Session()

model_s3_path = sagemaker_session.upload_data(
    path="model.tar.gz",
    bucket=bucket,
    key_prefix="ghcn-extreme/models"
)

print("Uploaded to:", model_s3_path)


Uploaded to: s3://sagemaker-us-east-1-083422367993/ghcn-extreme/models/model.tar.gz


In [47]:
predictor.delete_endpoint()
print("Endpoint deleted.")


Endpoint deleted.


In [93]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model_monitor import (
    DataCaptureConfig,
    DefaultModelMonitor
)
import pandas as pd
import time

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name
role = get_execution_role()

print("Bucket:", bucket)
print("Region:", region)


Bucket: sagemaker-us-east-1-083422367993
Region: us-east-1


In [27]:
project_prefix = "ghcn-extreme"
model_s3_path = f"s3://{bucket}/ghcn-extreme/models/model.tar.gz"

print("Model path:", model_s3_path)


Model path: s3://sagemaker-us-east-1-083422367993/ghcn-extreme/models/model.tar.gz


## Deploy Model Endpoint

The Random Forest model is selected as the production model
for monitoring demonstration.


In [73]:
from sagemaker.sklearn.model import SKLearnModel

model_s3_path = f"s3://{bucket}/ghcn-extreme/models/model.tar.gz"

sk_model = SKLearnModel(
    model_data=model_s3_path,
    role=role,
    entry_point="inference.py",
    framework_version="1.2-1",
    py_version="py3"
)

predictor = sk_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

print("Fresh endpoint deployed.")


------!Fresh endpoint deployed.


## Configure Model Monitor (Data Quality Monitoring)


In [78]:
from sagemaker.model_monitor import DatasetFormat

baseline_job = monitor.suggest_baseline(
    baseline_dataset=f"s3://{bucket}/ghcn-extreme/processed_csv/",
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=f"s3://{bucket}/{project_prefix}/monitoring/baseline",
    wait=True
)

print("Baseline statistics generated.")


INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2026-02-16-04-28-35-989


......................2026-02-16 04:32:17.945125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-02-16 04:32:17.945170: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2026-02-16 04:32:19.681499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2026-02-16 04:32:19.681540: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2026-02-16 04:32:19.681615: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-79-199.ec2.internal): /proc/driver/nvidia/version does not exist
2026-02-16 04:32:19.681948:

## Schedule Monitoring Job


In [95]:
import boto3
import sagemaker

boto_session = boto3.Session(region_name="us-east-1")
sm_session = sagemaker.Session(boto_session=boto_session)

from sagemaker.model_monitor import DefaultModelMonitor

monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=sm_session
)


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [83]:
from sagemaker.predictor import Predictor

endpoint_name = "sagemaker-scikit-learn-2026-02-16-04-10-18-956"

predictor = Predictor(endpoint_name=endpoint_name)

print("Attached to:", predictor.endpoint_name)


Attached to: sagemaker-scikit-learn-2026-02-16-04-10-18-956


In [99]:
from sagemaker.model_monitor import BatchTransformInput

batch_input = BatchTransformInput(
    data_captured_destination_s3_uri=f"s3://{bucket}/{project_prefix}/monitoring/batch_input",
    destination="/opt/ml/processing/input",
    dataset_format={
        "Csv": {
            "Header": False
        }
    },
    s3_data_distribution_type="FullyReplicated"
)

monitor.create_monitoring_schedule(
    monitor_schedule_name="ghcn-extreme-monitor-schedule",
    batch_transform_input=batch_input,
    output_s3_uri=f"s3://{bucket}/{project_prefix}/monitoring/reports",
    schedule_cron_expression="cron(0 * ? * * *)"
)

print("Monitoring schedule created.")


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: ghcn-extreme-monitor-schedule


Monitoring schedule created.


## Confirm Monitoring Schedule


In [100]:
monitor.describe_schedule()


{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:083422367993:monitoring-schedule/ghcn-extreme-monitor-schedule',
 'MonitoringScheduleName': 'ghcn-extreme-monitor-schedule',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2026, 2, 16, 4, 55, 0, 181000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2026, 2, 16, 4, 55, 3, 360000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2026-02-16-04-54-59-636',
  'MonitoringType': 'DataQuality'},
 'ResponseMetadata': {'RequestId': 'ecb5f59d-b0d9-4e66-aa9c-cb458915de8c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ecb5f59d-b0d9-4e66-aa9c-cb458915de8c',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control'

## Summary

The model endpoint is deployed with data capture enabled.
A baseline has been generated and a monitoring schedule created.

Model Monitor will now detect data quality deviations
and feature distribution shifts.
